# Feature Importance and Conclusion
___

#### Feature Importance
* 1.1 Feature importance selection
* 1.2 Retraining with the best parameters across each of the 5 CV folds.
* 1.3 Feature Importance - offer completion after viewing multiclass
* 1.4 Feature Importance - offer completion binary 

#### Results
* 2.1 Feature Importance Analysis
* 2.2 Results insights
* 2.3 Business Application
* 2.4 Model prediction examples

#### Conclusion
* 2.5 Conclusion and Reflection
* 2.6 Extensions to the model

## Feature Importance
___

Having chosen the best hyper parameters from our selection our next steps will be to analyse feature importances
* Can we remove features that do not add value to our model predictability and further improve accuracy?

In [1]:
# mount google drive if running in colab
import os
import sys

if os.path.exists('/usr/lib/python3.6/'):
    from google.colab import drive
    drive.mount('/content/drive/')
    sys.path.append('/content/drive/My Drive/Colab Notebooks/Starbucks_Udacity')
    %cd /content/drive/My Drive/Colab Notebooks/Starbucks_Udacity/notebooks/exploratory
else:
    sys.path.append('../../')

In [9]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import progressbar
import catboost
import joblib
from catboost import CatBoostClassifier
from catboost import Pool
from catboost import MetricVisualizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import timeit

from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, GroupKFold
from sklearn.model_selection import ParameterGrid

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import seaborn as sns

%load_ext autoreload
%autoreload 2
%aimport src.models.train_model
%aimport src.data.make_dataset

from src.data import make_dataset
from src.data.make_dataset import save_file
from src.models import train_model
from src.models.train_model import grid_search_results, generate_folds
from src.models.train_model import label_creater, exploratory_training

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1.1. Feature Importance Selection
First we will retrain each cross validation fold with the optimally selected hyper parameters.

CatBoost has various built in metrics to analyse feature importance. These include:

**PredictionValuesChange**
* Shows the average change in prediction value when a feature value changes. The greater the importance, the greater the unit change in prediction value.

**LossFunctionChange (Train or Test)**
* Represents the change in the loss value of a model with a particular feature included compared to without it. This would ideally be equivalent to a model retrained with the feature removed. Retraining the model with each feature <br> 
individually removed would be too computationally expensive but can be approximated by removing the relevant trees from the existing model. The calculation of LossFunctionChange is dependent on the data set used and can be measured by its affect against the train or test set.
    
We will consolidate each of these importance metrics for comparison and analysis.

In [20]:
def importances(model, train_pool, test_pool):
    '''
    Comparison of sorted feature imporTances based on:
    
    1. PredictionValuesChange
    2. LossFunctionChange train
    3. LossFunctionChange test
        
    Parameters
    -----------
    model: CatBoost model
    train_pool: CatBoost train data
    test_pool: CatBoost test data
               
    Returns
    -------
    DataFrame
    '''
    
    importance = pd.DataFrame(np.array(model.get_feature_importance(prettified=True)))
    loss_function_change = pd.DataFrame(
                                        np.array(model.get_feature_importance(
                                        train_pool,
                                        'LossFunctionChange',
                                        prettified=True)))
    loss_function_change_test = pd.DataFrame(
                                            np.array(model.get_feature_importance(
                                            test_pool,
                                            'LossFunctionChange',
                                            prettified=True)))
    
    df = pd.concat([importance, loss_function_change, loss_function_change_test], axis=1)
    df.columns = ['feature0', 'importance0', 'feature1', 'importance1', 'feature2', 'importance2']
    features = [x for x in df.columns if 'feature' in x]
    
    # separate by features and importances
    feature_list =[]
    for i, j in enumerate(features):
        feature_list.append(df[[j, 'importance'+str(i)]].set_index(j))
    
    # join by features as indedx
    features_joined = feature_list[0]
    for i in range(len(feature_list))[1:]:
        features_joined = features_joined.join(feature_list[i])
        
    return features_joined

### 1.2. Retraining with the best parameters across each of the 5 CV folds.

**Offer completion after viewing - Multiclass Classification**

    {'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.01}

**Offer completion - Binary Classification** <br>
(in this case the default CatBoost parameters)

    {'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.03}

In [19]:
def best_param_grid(grid_results_file, metric='logloss_mean', folds=5):
    '''
    Takes grid search resulTs file(s) and returns optimal parameters
    
    Parameters
    -----------
    grid_search_file: file name excluding extention and fold suffix
    folds: number of folds (or files) 
    metric: which metric to choose best parameters from, default ('logloss_mean')
    
    Returns
    -----------
    grid: dictionary of lists of parameters  
    '''  
    
    results_person, best_scores = grid_search_results(grid_results_file, 5, display_results=False)
    
    grid = best_scores[best_scores.Metric == metric + '_mean'].Params.values[0]
    
    for i in grid:
        grid[i] = [grid[i]]
        
    print('Training with ' + str(grid))
    
    return grid

In [18]:
def gridsearch_early_stopping_importance(cv, X, y, folds, grid, cat_features=[], save=None):
    '''
    Perform grid search with early stopping across each fold using 
    parameter grid, returning feature importance results.
    
    Parameters
    -----------
    cv: cross validation
    X: DataFrame or Numpy array
    y: DataFrame or Numpy array
    fold: list of fold indexes
    grid: parameter grid
    save:   string, excluding file extension (default=None)
            saves results_df for each fold to folder '../../data/interim'
            
    Returns
    -------
    list of DataFrames of importances per fold using metrics:
        > PredictionValuesChange
        > LossFunctionChange (train)
        > LossFunctionChange (test)                      
    '''
        
    # generate data folds
    train_X, train_y, test_X, test_y = generate_folds(cv, X, y)
       
    importance_folds = [0 for fold in folds]
    
    # iterate through specified folds
    for fold in folds:
        # assign train and test pools
        test_pool = Pool(data=test_X[fold], label=test_y[fold], cat_features=cat_features)
        train_pool = Pool(data=train_X[fold], label=train_y[fold], cat_features=cat_features)

        # creating results_df dataframe
        results_df = pd.DataFrame(columns=['params' + str(fold), 'logloss'+ str(fold), 
                                           'Accuracy'+ str(fold), 'iteration'+ str(fold)])

        best_score = 99999

        # iterate through parameter grid
        for params in ParameterGrid(grid):

            # create catboost classifer with parameter params
            model = CatBoostClassifier(cat_features=cat_features,
                                        early_stopping_rounds=50,
                                        task_type='GPU',
                                        custom_loss=['Accuracy'],
                                        iterations=7000,
                                        **params)
            # fit model
            model.fit(train_pool, eval_set=test_pool, verbose=400)
            importance_folds[fold] = importances(model, train_pool, test_pool)

    return(importance_folds)

In [17]:
def compare_importances(path='../../models/importances_multi.joblib'):
    '''
    Takes importances for each cross validation fold and calculates mean per importance metric.
    
    Parameters
    -----------
    path: path to importance_results
    method: string
            'importance1': PredictionValues_Change
            'importance2': LossFunctionChange Train
            'importance3': LossFunctionChange Test
            
    Returns
    -----------
    df: DataFrame    
    '''
    
    importance_type = {'importance0': 'PredictionValuesChange',
                      'importance1': 'LossFunctionChange_Train',
                      'importance2': 'LossFunctionChange_Test'}
    
    fold_result_list = joblib.load(path)
    folds = len(fold_result_list)
    
    importance_columns=[]
    
    for method in importance_type.keys():
           
        method_per_fold = [fold_result_list[i][method].rename(f'fold{i}') for i in range(folds)]

        df = pd.DataFrame(method_per_fold).transpose().rename\
            (columns={'importance2' : 'fold' + str(i) for i in range(folds)})
        df['mean'] = df.sum(axis=1)/folds
        df.sort_values('mean', ascending=False, inplace=True)
        df.rename(columns={'mean': importance_type[method]}, inplace=True)
        df.drop(['fold0', 'fold1', 'fold2', 'fold3', 'fold4'], axis=1, inplace=True)
        
        importance_columns.append(df)
        
    
    results = importance_columns[0].join(importance_columns[1]).join(importance_columns[2])
    
    return results    

### 1.3. Feature Importance -  offer completion after viewing - multiclass
Retrain each fold using optimal parameters and consolidating feature importances: <br>
**{'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.01}**

In [ ]:
complete_from_view = {'completed_not_viewed': 2, 
                    'completed_before_viewed': 2, 
                    'complete_anyway': 1,
                    'completed_responsive': 1,
                    'incomplete_responsive': 0,
                    'no_complete_no_view': 0,
                    'unresponsive': 0}

labelling = {'failed': 0, 'complete after':1, 'complete before':2}

In [58]:
df = joblib.load('../../data/interim/transcript_final_optimised.joblib')
df = label_creater(df, label_grid=complete)
cat_features = [0,4,5,92,93,94,95,96,97]
grid = best_param_grid('multiclass_gridsearch_inc10', metric='Accuracy', folds=5)

df.sort_values('time_days', inplace=True)

X = df.drop('label', axis=1)
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, 
                                                    random_state=42)
cv = TimeSeriesSplit(n_splits=5).split(X_train, y_train)

weights = [df.label.value_counts().sum() / df.label.value_counts()[i] for 
           i in range(0, df.label.nunique())]

importance_results = gridsearch_early_stopping_importance(cv, X_train, y_train, [0,1,2,3,4], 
                                                          grid, cat_features=cat_features)

Training with {'depth': [7], 'l2_leaf_reg': [1], 'learning_rate': [0.01]}
positive classification % per fold and length
train[0] 0.3988 (10171,)
test[0]  0.4224 (10170,)
train[1] 0.4106 (20341,)
test[1]  0.4496 (10170,)
train[2] 0.4236 (30511,)
test[2]  0.4618 (10170,)
train[3] 0.4331 (40681,)
test[3]  0.4679 (10170,)
train[4] 0.4401 (50851,)
test[4]  0.4674 (10170,)
0:	learn: 0.6829140	test: 0.6831325	best: 0.6831325 (0)	total: 86.2ms	remaining: 10m 3s
400:	learn: 0.4177062	test: 0.4260632	best: 0.4260632 (400)	total: 36.3s	remaining: 9m 57s
800:	learn: 0.4100662	test: 0.4239917	best: 0.4239917 (800)	total: 2m 34s	remaining: 19m 57s
1200:	learn: 0.4040011	test: 0.4232440	best: 0.4232440 (1200)	total: 4m 18s	remaining: 20m 46s
1600:	learn: 0.3988598	test: 0.4229329	best: 0.4229285 (1588)	total: 5m 42s	remaining: 19m 15s
2000:	learn: 0.3963069	test: 0.4227091	best: 0.4227073 (1995)	total: 6m 43s	remaining: 16m 48s
bestTest = 0.4226940354
bestIteration = 2132
Shrink model to first 2133 i

In [64]:
#joblib.dump(importance_results, '../../models/importances_multi.joblib', compress=True)

['../../models/importances_multi.joblib']

In [27]:
importance_results = compare_importances(path='../../models/importances_multi.joblib')

#### The following table shows the mean feature importance for each metric across the 5 training folds.
* The features have been sorted according to PredictionValuesChange.
* It can be seen that although the ranking of feature importance is similar between metrics, it is not identical.

In [28]:
importance_results

,PredictionValuesChange,LossFunctionChange_Train,LossFunctionChange_Test
id,15.787749,2.094488e-02,2.172642e-02
person,15.064215,2.402877e-02,8.403749e-03
reward,7.346407,4.066818e-03,1.299620e-03
difficulty,6.441851,3.271102e-03,-2.222130e-04
informational,5.726569,2.040888e-03,-1.971771e-04
hist_failed_complete,5.396880,6.881935e-03,2.193669e-03
income,5.369861,7.481871e-03,9.179094e-03
signed_up,4.964405,3.526241e-02,4.187127e-02
duration,3.907792,1.601609e-03,-5.557545e-04
hist_previous_offers,3.516153,3.265514e-03,-1.182180e-03


### 1.4 Feature Importance - offer completion binary
##### Retrain each fold using optimal parameters and consolidating feature importances:
**{'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.03}**

In [ ]:
complete = {'completed_not_viewed': 1, 
        'completed_before_viewed': 1, 
        'complete_anyway': 1,
        'completed_responsive': 1,
        'incomplete_responsive': 0,
        'no_complete_no_view': 0,
        'unresponsive': 0}

labelling = {'failed':0, 'completed':1}

In [296]:
df = joblib.load('../../data/interim/transcript_final_optimised.joblib')
df = label_creater(df, label_grid=complete)
cat_features = [0,4,5,92,93,94,95,96,97]
grid = best_param_grid('complete_gridsearch', metric='Accuracy', folds=5)

grid = {'depth': [6], 'l2_leaf_reg': [3], 'learning_rate': [0.03]}
df.sort_values('time_days', inplace=True)

X = df.drop('label', axis=1)
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, 
                                                    random_state=42)
cv = TimeSeriesSplit(n_splits=5).split(X_train, y_train)

weights = [df.label.value_counts().sum() / df.label.value_counts()[i] for 
           i in range(0, df.label.nunique())]

importance_results = gridsearch_early_stopping_importance(cv, X_train, y_train, [0,1,2,3,4], 
                                                          grid, cat_features=cat_features)

Training with {'depth': [9], 'l2_leaf_reg': [1], 'learning_rate': [0.03]}
positive classification % per fold and length
train[0] 0.3988 (10171,)
test[0]  0.4224 (10170,)
train[1] 0.4106 (20341,)
test[1]  0.4496 (10170,)
train[2] 0.4236 (30511,)
test[2]  0.4618 (10170,)
train[3] 0.4331 (40681,)
test[3]  0.4679 (10170,)
train[4] 0.4401 (50851,)
test[4]  0.4674 (10170,)
0:	learn: 0.6660971	test: 0.6665875	best: 0.6665875 (0)	total: 83.2ms	remaining: 9m 41s
400:	learn: 0.4081959	test: 0.4226921	best: 0.4226854 (398)	total: 28.5s	remaining: 7m 48s
bestTest = 0.4225237851
bestIteration = 513
Shrink model to first 514 iterations.
0:	learn: 0.6641097	test: 0.6639001	best: 0.6639001 (0)	total: 83.1ms	remaining: 9m 41s
400:	learn: 0.3010469	test: 0.3583611	best: 0.3583590 (399)	total: 50.4s	remaining: 13m 49s
bestTest = 0.3576927823
bestIteration = 496
Shrink model to first 497 iterations.
0:	learn: 0.6613101	test: 0.6604171	best: 0.6604171 (0)	total: 502ms	remaining: 58m 35s
400:	learn: 0.21539

In [297]:
#joblib.dump(importance_results, '../../models/importances_binary.joblib', compress=True)

['../../models/importances_binary.joblib']

In [23]:
importance_results_binary = compare_importances(path='../../models/importances_binary.joblib')

In [24]:
importance_results_binary

,PredictionValuesChange,LossFunctionChange_Train,LossFunctionChange_Test
person,16.057824,3.015906e-02,9.258295e-03
id,13.295159,2.631872e-02,2.677952e-02
duration,9.354440,4.378910e-03,1.278548e-03
difficulty,7.960779,3.708665e-03,7.545011e-04
hist_failed_complete,7.344255,4.213410e-03,-3.448952e-03
income,5.899365,1.163289e-02,1.414292e-02
signed_up,4.944489,2.450283e-02,2.565063e-02
hist_difficulty_completed,3.937516,9.490014e-04,2.061426e-04
reward,3.619008,3.525652e-03,1.610468e-03
informational,3.434714,1.283225e-03,-1.483046e-03


### 2.1 Feature Importance Analysis

#### Mean importances per fold
* As expected, the original base features generally have the greatest influence on the model. 

In [29]:
print('Top 5 mean features per fold by importance')
top_5 = pd.DataFrame()

experiment = {'multi': importance_results, 'binary': importance_results_binary}

for i in ['multi', 'binary']:
    for j in importance_results.columns:
        top_5_column = pd.Series(experiment[i].nlargest(5, j)[j].index.values, name=f'{i}_{j}')
        top_5[f'{i} {j}'] = top_5_column
display(top_5)

Top 5 mean features per fold by importance


,multi PredictionValuesChange,multi LossFunctionChange_Train,multi LossFunctionChange_Test,binary PredictionValuesChange,binary LossFunctionChange_Train,binary LossFunctionChange_Test
0,id,signed_up,signed_up,person,person,id
1,person,person,id,id,id,signed_up
2,reward,id,hist_viewed_spend,duration,signed_up,income
3,difficulty,hist_viewed_spend,income,difficulty,hist_viewed_spend,hist_viewed_spend
4,informational,amount_per_day_not_offer,person,hist_failed_complete,income,amount_per_day_not_offer


The following metrics occured most frequently within the top5 importances by metric and experiment type:

In [415]:
top_5.melt().value.value_counts()

id                          6
person                      5
signed_up                   4
hist_viewed_spend           4
income                      3
amount_per_day_not_offer    2
difficulty                  2
informational               1
duration                    1
hist_failed_complete        1
reward                      1
Name: value, dtype: int64

* id (the type of offer given) was most frequently in the top 5. This indicates that the different offers are significantly different from each other with differing effects on different customers.
* person (the predominant feature representing unique properties of a given individual) occurred 5 times.
* hist_viewed_spend and amount_per_day_not_offer are the most common engineered features frequently appearing in the top 5. These represent the spending habits or responsiveness of a customer with and without an offer active.

We will now experiment with removing the weakest features from the model using each importance metric, varying the threshold percentage of features to remove.

E.g. first removing the worst 1% of features as determined by mean prediction value change per fold and retraining to determine accuracy.

#### Offer complete after view, feature removal training:

In [ ]:
complete_from_view = {'completed_not_viewed': 2, 
                    'completed_before_viewed': 2, 
                    'complete_anyway': 1,
                    'completed_responsive': 1,
                    'incomplete_responsive': 0,
                    'no_complete_no_view': 0,
                    'unresponsive': 0}

labelling = {'failed': 0, 'complete after':1, 'complete before':2}
params = {'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.01}
thresholds = [0.01, 0.05, 0.10, 0.20]
results = []

for i in importance_results.columns:
    for j in thresholds:
        
        to_remove = importance_results[importance_results[i] < 
                                       importance_results[i].quantile(q=j)].index
           
        accuracy = exploratory_training(labels=complete_from_view, drop_features=to_remove, 
                                        labels_compact=labelling, verbose=False, **params)
        
        results.append([i,j, accuracy])
        
joblib.dump(results, '../../models/importances_multi_results.joblib', compress=True)

In [30]:
importances_multi_results = joblib.load('../../models/importances_multi_results.joblib')

In [33]:
multi_importance_testing = pd.DataFrame(importances_multi_results, 
                                        columns=['Importance_type', 'Removal_threshold', 
                                                 'Accuracy'])

multi_importance_testing['vs all features'] = multi_importance_testing['Accuracy'] - 0.716636

print(f'Base accuracy: 0.716636')
multi_importance_testing.style.format({'vs all features': "{:.2%}"})

Base accuracy: 0.716636


,Importance_type,Removal_threshold,Accuracy,vs all features
0,PredictionValuesChange,0.01,0.71231,-0.43%
1,PredictionValuesChange,0.05,0.712441,-0.42%
2,PredictionValuesChange,0.1,0.71054,-0.61%
3,PredictionValuesChange,0.2,0.709688,-0.69%
4,LossFunctionChange_Train,0.01,0.711917,-0.47%
5,LossFunctionChange_Train,0.05,0.71408,-0.26%
6,LossFunctionChange_Train,0.1,0.713359,-0.33%
7,LossFunctionChange_Train,0.2,0.710212,-0.64%
8,LossFunctionChange_Test,0.01,0.710147,-0.65%
9,LossFunctionChange_Test,0.05,0.710212,-0.64%


* Removing the bottom 1% to 20% of features using the three importance metrics yielded the above accuracies. There was a fall in accuracy in each case vs using all features, indicating that the features grouped in the thresholds removed are contributing a positive impact to model accuracy.


#### Offer completion binary, feature removal training

In [ ]:
complete = {'completed_not_viewed': 1, 
                    'completed_before_viewed': 1, 
                    'complete_anyway': 1,
                    'completed_responsive': 1,
                    'incomplete_responsive': 0,
                    'no_complete_no_view': 0,
                    'unresponsive': 0}

labelling = {'failed': 0, 1: 'complete'}
params = {'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.03}
thresholds = [0.01, 0.05, 0.10, 0.20]
results = []

for i in importance_results_binary.columns:
    for j in thresholds:
        
        to_remove = importance_results_binary[importance_results_binary[i] < 
                                              importance_results_binary[i].quantile(q=j)].index
           
        accuracy = exploratory_training(labels=complete, drop_features=to_remove, 
                                        labels_compact=labelling, verbose=False, **params)
        
        results.append([i,j, accuracy])
        
joblib.dump(results, '../../models/importances_binary_results.joblib', compress=True)

In [39]:
importances_binary_results = joblib.load('../../models/importances_binary_results.joblib')

In [50]:
binary_importance_testing = pd.DataFrame(importances_binary_results, 
                                         columns=['Importance_type', 'Removal_threshold', 
                                                  'Accuracy'])

binary_importance_testing['vs all features'] = binary_importance_testing['Accuracy'] - 0.851206

print(f'Base accuracy: 0.851206')
binary_importance_testing.style.format({'vs all features': "{:.2%}"})

Base accuracy: 0.851206


,Importance_type,Removal_threshold,Accuracy,vs all features
0,PredictionValuesChange,0.01,0.847666,-0.35%
1,PredictionValuesChange,0.05,0.850616,-0.06%
2,PredictionValuesChange,0.1,0.85042,-0.08%
3,PredictionValuesChange,0.2,0.847142,-0.41%
4,LossFunctionChange_Train,0.01,0.849109,-0.21%
5,LossFunctionChange_Train,0.05,0.851206,0.00%
6,LossFunctionChange_Train,0.1,0.848715,-0.25%
7,LossFunctionChange_Train,0.2,0.847994,-0.32%
8,LossFunctionChange_Test,0.01,0.847798,-0.34%
9,LossFunctionChange_Test,0.05,0.850616,-0.06%


* Again for the binary model, when we remove varying thresholds of the weakest features we see a drop in the overall accuracy, indicating that the features that have been engineered make a positive contribution to the model.
* On the other hand, it was possible to remove up to 20% of the weakest features and only reduce accuracy by 0.17% for the Binary classification experiment and 0.64% for the Multi class classification experiment using LossFunctionChange Test as a metric. This could be due to a high degree of correlation between features, with removal of some features providing minimal reduction in accuracy as the decision trees utilise other features to take up the slack. 
* If training time was an issue for a particularly large dataset it might be useful to sacrifice accuracy by removing weak features to improve training time. 

### 2.2 Results insights
* These results show the strength of the gradient boosting algorithm to solve this kind of problem.
* In this case we used CatBoost, but other algorithms such as XGBoost or LightGBM could also have been used and we would expect similarly strong results.
* Many of the features created were highly correlated with each other. One of the strengths in particular of gradient boosted decision trees is the ability to handle and ignore correlation between features. Using a method such as linear regression on the other hand would have been problematic with highly correlated features.

### 2.3 Business Application

Now that we have established the best models for these two classification experiments we can move our focus to predicting the probability that any offer would be completed. 

In [5]:
complete = {'completed_not_viewed': 1, 
                    'completed_before_viewed': 1, 
                    'complete_anyway': 1,
                    'completed_responsive': 1,
                    'incomplete_responsive': 0,
                    'no_complete_no_view': 0,
                    'unresponsive': 0}
labelling = {'failed': 0, 1: 'complete'}

binary_model, X_test, y_test = exploratory_training(labels=complete, 
                                                    labels_compact=labelling, verbose=False, 
                                                    **{'depth': 6, 'l2_leaf_reg': 3, 
                                                    'learning_rate': 0.03}, return_model=True)

joblib.dump(binary_model, '../../models/binary_model.joblib', compress=True)

['../../models/binary_model.joblib']

We will use the simple binary complete/fail model that achieved 85.1% accuracy. The following function takes an observation within the Test data set and determines the probabilities that the customer will complete or fail to complete for each of the 10 possible offers.

In [538]:
def testing_offers(observation, X_test):
    '''
    Utilises binary_model along with X_test data to determine the
    probabilities that each observed offer would have been completed if
    any of the 10 possible offers were given, all other factors 
    remaining equal.
    Offer and demographic data have been added to the output DataFrame.
    
    Parameters
    -----------
    obervation: int position with X_test
    X_test: DataFrame of test Data
            
    Returns
    -----------
    df: DataFrame    
    '''
    
    binary_model = joblib.load('../../models/binary_model.joblib')
         
    # observation - row data within test set
    row = X_test.iloc[observation:observation+1,:].copy()
    
    probabilities=[]
    
    # the actual offer that was given in the test set
    actual = row.id.values[0]
    
    # loop through 10 possible offers and calculate probabilities using 
    # model    
    for i in range(10):
        row.id=i               
        if row.id.values[0] == actual:
            original = binary_model.predict_proba(row)[0].tolist()
            original.extend([row.id.values[0]])
            probabilities.append(original)
        else:
            probabilities.append(binary_model.predict_proba(row)[0])
    
    # create DataFrame of results    
    results = pd.DataFrame(probabilities)
    
    # determine if the model correctly identified the actual offer
    if y_test.iloc[observation] == results[results[2] > 1].max()[:-1].idxmax():
        results[2][actual] = 'correct'
    else:
        results[2][actual] = 'incorrect'
    
    # rename columns and compute total complete probability
    results.columns=['failed', 'complete', 'prediction']
   
    
    # join dataframe to offer id information
    portw = joblib.load('../../data/interim/portw.joblib')
    results = results.join(portw)
       
    results['age'] = row.age.values[0]
    results['income'] = row.income.values[0]
    results['gender'] = row.gender.values[0]
           
    return results

### 2.4 Model prediction examples

* In the following output we can see that observation 1 was correctly predicted as failing to complete offer c.
* Failed and complete columns show the model's predicted probabilities for each of the possible offers, all other features being equal.

In [523]:
testing_offers(1, X_test)

,failed,complete,prediction,id,difficulty,reward,duration,mobile,web,social,bogo,discount,informational,age,income,gender
0,0.969962,0.030038,NaN,a,20,5,10,0,1,0,0,1,0,56.0,48000.0,1
1,0.970268,0.029732,NaN,b,10,10,7,1,0,1,1,0,0,56.0,48000.0,1
2,0.976791,0.023209,correct,c,10,10,5,1,1,1,1,0,0,56.0,48000.0,1
3,0.848571,0.151429,NaN,d,10,2,10,1,1,1,0,1,0,56.0,48000.0,1
4,0.943322,0.056678,NaN,e,10,2,7,1,1,0,0,1,0,56.0,48000.0,1
5,0.835954,0.164046,NaN,f,7,3,7,1,1,1,0,1,0,56.0,48000.0,1
6,0.901919,0.098081,NaN,g,5,5,7,1,1,0,1,0,0,56.0,48000.0,1
7,0.901970,0.098030,NaN,h,5,5,5,1,1,1,1,0,0,56.0,48000.0,1
8,0.989400,0.010600,NaN,i,0,0,4,1,1,0,0,0,1,56.0,48000.0,1
9,0.989377,0.010623,NaN,j,0,0,3,1,0,1,0,0,1,56.0,48000.0,1


* In the next example, observation 4 was correctly predicted as failing to complete offer i.

In [524]:
testing_offers(4, X_test)

,failed,complete,prediction,id,difficulty,reward,duration,mobile,web,social,bogo,discount,informational,age,income,gender
0,0.911059,0.088941,NaN,a,20,5,10,0,1,0,0,1,0,69.0,51000.0,0
1,0.910220,0.089780,NaN,b,10,10,7,1,0,1,1,0,0,69.0,51000.0,0
2,0.953317,0.046683,NaN,c,10,10,5,1,1,1,1,0,0,69.0,51000.0,0
3,0.171935,0.828065,NaN,d,10,2,10,1,1,1,0,1,0,69.0,51000.0,0
4,0.788004,0.211996,NaN,e,10,2,7,1,1,0,0,1,0,69.0,51000.0,0
5,0.216156,0.783844,NaN,f,7,3,7,1,1,1,0,1,0,69.0,51000.0,0
6,0.576276,0.423724,NaN,g,5,5,7,1,1,0,1,0,0,69.0,51000.0,0
7,0.605231,0.394769,NaN,h,5,5,5,1,1,1,1,0,0,69.0,51000.0,0
8,0.996723,0.003277,correct,i,0,0,4,1,1,0,0,0,1,69.0,51000.0,0
9,0.996717,0.003283,NaN,j,0,0,3,1,0,1,0,0,1,69.0,51000.0,0


* In the following example we incorrectly classified offer a as failing to complete

In [525]:
testing_offers(8, X_test)

,failed,complete,prediction,id,difficulty,reward,duration,mobile,web,social,bogo,discount,informational,age,income,gender
0,0.221816,0.778184,incorrect,a,20,5,10,0,1,0,0,1,0,55.0,35000.0,0
1,0.221003,0.778997,NaN,b,10,10,7,1,0,1,1,0,0,55.0,35000.0,0
2,0.252728,0.747272,NaN,c,10,10,5,1,1,1,1,0,0,55.0,35000.0,0
3,0.046379,0.953621,NaN,d,10,2,10,1,1,1,0,1,0,55.0,35000.0,0
4,0.171979,0.828021,NaN,e,10,2,7,1,1,0,0,1,0,55.0,35000.0,0
5,0.050354,0.949646,NaN,f,7,3,7,1,1,1,0,1,0,55.0,35000.0,0
6,0.094381,0.905619,NaN,g,5,5,7,1,1,0,1,0,0,55.0,35000.0,0
7,0.093106,0.906894,NaN,h,5,5,5,1,1,1,1,0,0,55.0,35000.0,0
8,0.654059,0.345941,NaN,i,0,0,4,1,1,0,0,0,1,55.0,35000.0,0
9,0.653659,0.346341,NaN,j,0,0,3,1,0,1,0,0,1,55.0,35000.0,0


For any observation, we now have predicted probabilities that any offer will complete. Depending on the business targets we can use these to make a decision process of how to target any customer at any point in time given their past history.

### 2.5 Conclusion and Reflection

* The raw dataset being initially organised in a non tidy format proved a challenge with the key question being what would be the best method to organise the data before being able to train a model. 
* Since historical data was provided across multiple rows (in particular the extensive transactional data) if was difficult to capture as much of this information as possible and convert it into a single observation point.
* Feature engineering proved to be the strongest contribution to improving model accuracy and was challenging since the functions required nested looping across the whole 300,000 line dataset. I initially wrote these functions looping over Pandas DataFrame and Series data structures. Each function would range from taking 10 minutes to up to two hours to run. I quickly realised that this was creating significant time overhead, even using the Pandas function itterows which is designed for this purpose. Converting the data to numpy arrays before looping and making any calculations sped up the processes significantly.
* Unexpectedly there was very minor improvement in accuracy by tuning hyper parameters using grid search and cross validation. This took significant computational time with relatively low reward. As mentioned earlier, this could be due to the highly optimised default parameter settings of CatBoost. There are however an extensive list of possible hyper parameters that can be tuned with CatBoost and further explored. For this project I focused on the most common hyper parameters and those that I believed would have yielded the greatest accuracy improvements. If I was to extend the experiment and explore optimising further hyper parameters, I would not perform a brute force gridsearch due to the extensive time required, but instead investigate Bayesian hyper parameter optimisation. 
* Creating my own grid search function allowing for early stopping when test logloss increased was extremely useful. The optimal number of iterations varied widely over different depth and learning rate combinations and without early stopping it would have been very difficult to optimise the number of iterations required. Since I was using a GPU for model training, I was not able to parallelise my gridsearch. It is possible that running gridsearch using the CPU across multiple threads could have been faster overall and this would be something to investigate going forward.
* There were a large number of experiments that needed to be run in varying combinations of:
    * hyper parameters
    * target labels
    * cross validation folds
    * feature removal combinations<br>

Constructing optimal pipelines created from these combinations quickly and effectively was one of the biggest challenges and I refactored this code multiple times. A key insight was that writing functions/pipelines to loop over all the possible feature/parameter/label combinations to be investigated and producing a clearly summarised set of results in one go was vastly superior to manually running individual model training, tweaking parameters and then running again multiple times. The latter method resulted in messy convoluted notebooks and reduced clarity. Furthermore, despite the significant compute time required to run multiple training combinations in a row, I was able to utilise google colab for any extensive lengthy model training and therefore avoid draining resources on my local machine.

No doubt the design architecture could be significantly improved upon and this will be one of my major focuses as I seek to develop in the field.

Whilst currently gradient boosting decision trees tend to be the best algorithms at quickly providing highly accurate models for tabular data, I am interested to explore how neural network architectures can also be applied effectively. In particular, stacking neural networks with gradient boosted decision trees should be able to provide further accuracy improvements.

### 2.6 Possible Extensions to the Model
* I explored trying to calculate metrics to determine the choice of offer that would maximise revenue in each instance. Whilst predicting probability that any offer would be completed proved accurate at 85.1%, optimising for revenue did not prove accurately feasible using my current classification models. This is because they required too much emphasis on establishing and calculating a base rate of spending by the customer via feature engineering. As earlier discussed, this was not accurate due to the limited amount of time the model was run and observations only ranged from 0 to 30 days time period.
* To develop the model further I would train a CatBoost Regressor with target labels of spending under the influence of an offer and whilst not under the influence of the offer. Since the resulting prediction would give an actual spending value instead of a True/False or probability of completion, this would be much more effective to use to optimise revenue.